In [1]:
import pandas as pd
import json
from transformers import pipeline

In [2]:
df = pd.read_excel('04_Consolidado.xlsx')
df['eval'] = None

In [3]:
# Load a sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# 1 star: Very negative sentiment.
# 2 stars: Negative sentiment.
# 3 stars: Neutral sentiment.
# 4 stars: Positive sentiment.
# 5 stars: Very positive sentiment.

c:\Users\waldo\miniforge3\envs\uni\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# for index, row in df.iterrows():
#     if row['eval'] is not None:
#         print(f'Skipped: {index}')
#         continue

#     try:
#         oraciones = pd.DataFrame({'oracion' : json.loads(row['oraciones'])})
#         oraciones['etiqueta'] = None
#         oraciones['puntaje'] = None
#         oraciones['estrellas'] = None
#     except:
#         continue
    
#     for ind, oracion in oraciones.iterrows():
#         result = classifier(oracion['oracion'])
#         oraciones.at[ind,'etiqueta'] = result[0]['label']
#         oraciones.at[ind,'estrellas'] = int(result[0]['label'][0])
#         oraciones.at[ind,'puntaje'] = float(result[0]['score'])

#     df.loc[index,'eval'] = oraciones.to_json()
#     print(f'Wrote: {index}', end='\r')

In [6]:
from concurrent.futures import ThreadPoolExecutor

In [7]:
def process_row(index, row):
    if row['eval'] is not None:
        print(f'Skipped: {index}')
        return

    try:
        oraciones = pd.DataFrame({'oracion' : json.loads(row['oraciones'])})
        oraciones['etiqueta'] = None
        oraciones['puntaje'] = None
        oraciones['estrellas'] = None
    except:
        return
    
    for ind, oracion in oraciones.iterrows():
        result = classifier(oracion['oracion'])
        oraciones.at[ind,'etiqueta'] = result[0]['label']
        oraciones.at[ind,'estrellas'] = int(result[0]['label'][0])
        oraciones.at[ind,'puntaje'] = float(result[0]['score'])

    df.loc[index,'eval'] = oraciones.to_json()
    print(f'Wrote: {index}', end='\r')

In [8]:
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_row, index, row) for index, row in df.iterrows()]
    for future in futures:
        future.result()

Skipped: 0
Skipped: 1
Skipped: 2
Skipped: 3
Skipped: 4
Skipped: 5
Skipped: 6
Skipped: 7
Skipped: 8
Skipped: 9
Skipped: 10
Skipped: 11
Skipped: 12
Skipped: 13
Skipped: 14
Skipped: 15
Skipped: 16
Skipped: 17
Skipped: 18
Skipped: 19
Skipped: 20
Skipped: 21
Skipped: 22
Skipped: 23
Skipped: 24
Skipped: 25
Skipped: 26
Skipped: 27
Skipped: 28
Skipped: 29
Skipped: 30
Skipped: 31
Skipped: 32
Skipped: 33
Skipped: 34
Skipped: 35
Skipped: 36
Skipped: 37
Skipped: 38
Skipped: 39
Skipped: 40
Skipped: 41
Skipped: 42
Skipped: 43
Skipped: 44
Skipped: 45
Skipped: 46
Skipped: 47
Skipped: 48
Skipped: 49
Skipped: 50
Skipped: 51
Skipped: 52
Skipped: 53
Skipped: 54
Skipped: 55
Skipped: 56
Skipped: 57
Skipped: 58
Skipped: 59
Skipped: 60
Skipped: 61
Skipped: 62
Skipped: 63
Skipped: 64
Skipped: 65
Skipped: 66
Skipped: 67
Skipped: 68
Skipped: 69
Skipped: 70
Skipped: 71
Skipped: 72
Skipped: 73
Skipped: 74
Skipped: 75
Skipped: 76
Skipped: 77
Skipped: 78
Skipped: 79
Skipped: 80
Skipped: 81
Skipped: 82
Skipped: 83
Sk

Token indices sequence length is longer than the specified maximum sequence length for this model (563 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (563) must match the size of tensor b (512) at non-singleton dimension 1

In [9]:
df.to_excel('05_Sentimiento.xlsx')